# 1. Cài đặt Thư viện

In [100]:
# Cell 1: Nâng cấp pip và các thư viện cần thiết
!pip install --upgrade pip
!pip install -U numpy torch torchvision transformers datasets acceleratehuggingface-hub
!pip install trl==0.11.3

# Import các thư viện để kiểm tra
import torch
import transformers
import datasets
import trl

# Hiển thị phiên bản của các thư viện chính
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"Datasets version: {datasets.__version__}")
print(f"TRL version: {trl.__version__}")

# Kiểm tra GPU
if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available, using CPU")

ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement acceleratehuggingface-hub (from versions: none)
ERROR: No matching distribution found for acceleratehuggingface-hub
PyTorch version: 2.7.1+cu126
Transformers version: 4.52.4
Datasets version: 3.6.0
TRL version: 0.18.1
GPU available: Tesla T4


# Cell 2: Tải dữ liệu Anthropic/hh-rlhf

In [7]:
from datasets import load_dataset

# Tải dataset
print("Đang tải dataset Anthropic/hh-rlhf...")
dataset = load_dataset("Anthropic/hh-rlhf")

# Lấy một tập con lớn hơn nhưng vẫn trong khả năng xử lý
total_samples = 100000  # Lấy 100000 mẫu đầu tiên
train_ratio = 0.8
train_size = int(total_samples * train_ratio)
test_size = total_samples - train_size

# Lấy mẫu tuần tự từ đầu dataset
train_data = dataset['train'].select(range(train_size))  # Lấy 80000 mẫu đầu cho training
test_data = dataset['train'].select(range(train_size, total_samples))  # 20000 mẫu tiếp theo cho testing

print(f"\nĐã chuẩn bị:")
print(f"- {len(train_data)} mẫu cho training ({train_ratio*100}%)")
print(f"- {len(test_data)} mẫu cho testing ({(1-train_ratio)*100}%)")
print(f"- Tổng số mẫu: {total_samples} (từ tổng số {len(dataset['train'])} mẫu có sẵn)")

Đang tải dataset Anthropic/hh-rlhf...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Đã chuẩn bị:
- 80000 mẫu cho training (80.0%)
- 20000 mẫu cho testing (19.999999999999996%)
- Tổng số mẫu: 100000 (từ tổng số 160800 mẫu có sẵn)


# Cell 3: Chuẩn bị mô hình cơ sở

In [8]:
# Cell 3: Initialize base model
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Use GPT-2
model_name = "gpt2"
print(f"Loading {model_name} model...")

try:
    # Check GPU and set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    if torch.cuda.is_available():
        print(f"GPU Memory before loading model: {torch.cuda.memory_allocated()/1024**2:.2f} MB")

    # Load model and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    model = GPT2LMHeadModel.from_pretrained(
        model_name,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )
    model = model.to(device)

    if torch.cuda.is_available():
        print(f"GPU Memory after loading model: {torch.cuda.memory_allocated()/1024**2:.2f} MB")

    # Add special tokens
    special_tokens = {"pad_token": "[PAD]"}
    num_added_toks = tokenizer.add_special_tokens(special_tokens)
    model.resize_token_embeddings(len(tokenizer))

    print("Base model loaded successfully!")

    # Test generation
    test_input = "What are some effective study techniques?"
    # Tokenize with attention mask
    inputs = tokenizer(
        test_input,
        return_tensors="pt",
        padding=True,
        return_attention_mask=True
    ).to(device)

    with torch.no_grad():
        output = model.generate(
            inputs.input_ids,
            attention_mask=inputs.attention_mask,
            max_new_tokens=50,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.pad_token_id
        )

    # Chỉ lấy phần text mới được sinh ra
    input_length = len(tokenizer.encode(test_input))
    generated_tokens = output[0][input_length:]
    output_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    print(f"\nTest generation - Input: '{test_input}'")
    print(f"Generated output: '{output_text}'")

except Exception as e:
    print(f"Error loading model: {str(e)}")
    print("Additional error info:", e.__class__.__name__)
    import traceback
    print(traceback.format_exc())

Loading gpt2 model...
Using device: cuda
GPU Memory before loading model: 0.00 MB


The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


GPU Memory after loading model: 255.49 MB
Base model loaded successfully!

Test generation - Input: 'What are some effective study techniques?'
Generated output: '

The results of this study are presented in this issue of the journal, "Science," and will be published in the journal Science in the next few months.

What are some of the limitations of this study?

This study is'


# Cell 4: Chuẩn bị dữ liệu ví dụ cho huấn luyện

In [9]:
# Cell 4: Chuẩn bị mẫu cho việc huấn luyện
demo_samples = []

for i in range(len(train_data)):
    sample = train_data[i]
    chosen = sample['chosen']
    rejected = sample['rejected']

    # Tìm vị trí Assistant cuối cùng trong cuộc hội thoại
    chosen_parts = chosen.split("\n\nAssistant: ")
    if len(chosen_parts) > 1:
        # Tất cả nội dung trước Assistant cuối cùng sẽ là prompt
        prompt = "\n\nAssistant: ".join(chosen_parts[:-1]) + "\n\nAssistant: "
        chosen_completion = chosen_parts[-1]
    else:
        # Trường hợp không tìm thấy "Assistant: ", ghi log để kiểm tra
        print(f"Warning: Không tìm thấy 'Assistant: ' trong mẫu {i}")
        continue  # Bỏ qua mẫu này

    # Tương tự với phần rejected
    rejected_parts = rejected.split("\n\nAssistant: ")
    if len(rejected_parts) > 1:
        rejected_completion = rejected_parts[-1]
    else:
        print(f"Warning: Không tìm thấy 'Assistant: ' trong phần rejected của mẫu {i}")
        continue  # Bỏ qua mẫu này

    demo_samples.append({
        "prompt": prompt,
        "chosen": chosen_completion,
        "rejected": rejected_completion
    })

# Hiển thị kết quả
num_display = 5  # Số mẫu muốn hiển thị để kiểm tra
# Thêm thống kê về số mẫu bị bỏ qua
print(f"Số mẫu bị bỏ qua: {len(train_data) - len(demo_samples)}")
print(f"\nĐã xử lý {len(demo_samples)} mẫu. Chỉ hiển thị {min(num_display, len(demo_samples))} mẫu đầu để kiểm tra.")

# Hiển thị một số mẫu để kiểm tra
for i in range(min(num_display, len(demo_samples))):
    print(f"\nMẫu số {i+1}/{len(demo_samples)}:")
    print("=" * 80)
    print("PROMPT (bao gồm tất cả các lượt trong cuộc hội thoại):")
    print(f"{demo_samples[i]['prompt']}")
    print("-" * 80)
    print("CÂU TRẢ LỜI ĐƯỢC CHỌN (CHOSEN) (phản hồi cuối cùng của Assistant):")
    print(f"{demo_samples[i]['chosen']}")
    print("-" * 80)
    print("CÂU TRẢ LỜI BỊ TỪ CHỐI (REJECTED) (phản hồi cuối cùng của Assistant):")
    print(f"{demo_samples[i]['rejected']}")
    print("=" * 80)

Số mẫu bị bỏ qua: 0

Đã xử lý 80000 mẫu. Chỉ hiển thị 5 mẫu đầu để kiểm tra.

Mẫu số 1/80000:
PROMPT (bao gồm tất cả các lượt trong cuộc hội thoại):


Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,

Human: What's your favorite one?

Assistant: 
--------------------------------------------------------------------------------
CÂU TRẢ LỜI ĐƯỢC CHỌN (CHOSEN) (phản hồi cuối cùng của Assistant):
I haven't even thought about it.
--------------------------------------------------------------------------------
CÂU TRẢ LỜI BỊ TỪ CHỐI (R

# 5.Xây dựng Reward Function

In [20]:
# Cell 5: Xây dựng Reward Function
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

# Load một model phù hợp cho việc tính embeddings
print("Đang tải model để tính embeddings...")
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embedding_model = AutoModel.from_pretrained(embedding_model_name).to(device)
embedding_tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)

def get_embeddings(text, model=embedding_model, tokenizer=embedding_tokenizer):
    """Lấy embeddings của văn bản sử dụng model chuyên biệt"""
    # Tokenize và truncate văn bản
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=512,
        padding=True
    ).to(device)

    # Lấy embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        # Lấy [CLS] token embedding hoặc mean của all tokens
        embeddings = outputs.last_hidden_state.mean(dim=1)

    return embeddings

def compute_similarity_score(text1, text2):
    """Tính cosine similarity giữa hai đoạn văn bản"""
    emb1 = get_embeddings(text1)
    emb2 = get_embeddings(text2)
    similarity = F.cosine_similarity(emb1, emb2)
    return similarity.item()

def reward_function(response, chosen, rejected):
    """Reward function dựa trên so sánh với chosen và rejected responses"""
    try:
        # Tính similarity với chosen và rejected
        chosen_similarity = compute_similarity_score(response, chosen)
        rejected_similarity = compute_similarity_score(response, rejected)

        # Normalize similarities to [0, 1]
        chosen_similarity = (chosen_similarity + 1) / 2
        rejected_similarity = (rejected_similarity + 1) / 2

        # Reward cao nếu giống chosen và khác rejected
        reward = chosen_similarity - 0.5 * rejected_similarity

        return max(0.0, min(1.0, reward))  # Clip reward về khoảng [0, 1]
    except Exception as e:
        print(f"Lỗi khi tính reward: {str(e)}")
        return 0.5  # Giá trị mặc định khi có lỗi

# Test reward function với một số mẫu từ demo_samples
print("\nKiểm tra reward function với các mẫu đầu tiên từ demo_samples:")
for i in range(min(5, len(demo_samples))):
    sample = demo_samples[i]
    print(f"\nMẫu {i+1}:")
    print("-" * 40)
    print(f"Prompt: {sample['prompt']}")
    print(f"Chosen: {sample['chosen']}")
    print(f"Rejected: {sample['rejected']}")

    # Test với chosen làm response (should get high reward)
    chosen_reward = reward_function(sample['chosen'], sample['chosen'], sample['rejected'])
    # Test với rejected làm response (should get low reward)
    rejected_reward = reward_function(sample['rejected'], sample['chosen'], sample['rejected'])

    print(f"\nReward khi response = chosen: {chosen_reward:.4f}")
    print(f"Reward khi response = rejected: {rejected_reward:.4f}")

print("\nĐã xây dựng và kiểm tra reward function!")

Đang tải model để tính embeddings...

Kiểm tra reward function với các mẫu đầu tiên từ demo_samples:

Mẫu 1:
----------------------------------------
Prompt: 

Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,

Human: What's your favorite one?

Assistant: 
Chosen: I haven't even thought about it.
Rejected: Ass.

Reward khi response = chosen: 0.7149
Reward khi response = rejected: 0.0702

Mẫu 2:
----------------------------------------
Prompt: 

Human: What kind of noises did dinosaurs make?

Assistant: Humans and dinosaurs didn’t

# 6. PPO

In [22]:
# Cell 6: PPO (Proximal Policy Optimization)
from trl import PPOConfig, PPOTrainer, AutoModelForCausalLMWithValueHead
import time
import torch

# Cấu hình PPO
print("Khởi tạo PPO config...")
ppo_config = PPOConfig(
    batch_size=32,  # tăng batch size lên để tối ưu hóa training
    mini_batch_size=4,
    learning_rate=1e-5,
    log_with=None,
    ppo_epochs=3,
    seed=42,
    gradient_accumulation_steps=1
)

print("Chuyển đổi model sang dạng ValueHead...")
# Dùng AutoModelWithValueHead để thêm value head cho PPO
ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)
ppo_model.to(device)

print("Khởi tạo PPO trainer...")
# Tạo PPO trainer
ppo_trainer = PPOTrainer(
    model=ppo_model,
    config=ppo_config,
    tokenizer=tokenizer,
)

def format_time(seconds):
    """Format thời gian từ giây sang dạng readable"""
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"

# Hàm để chạy PPO demo
def run_ppo_demo(samples, num_epochs=1):
    print("\nBắt đầu huấn luyện PPO...")
    logs = {
        "reward": [],
        "policy_loss": [],
        "value_loss": []
    }

    total_start_time = time.time()

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        epoch_start_time = time.time()

        # Xử lý theo batches
        for i in range(0, len(samples), ppo_config.batch_size):
            batch_start_time = time.time()
            batch_samples = samples[i:i + ppo_config.batch_size]

            query_tensors = []
            response_tensors = []
            rewards = []

            # Xử lý từng mẫu trong batch
            for sample in batch_samples:
                prompt, chosen, rejected = sample["prompt"], sample["chosen"], sample["rejected"]

                # Tokenize prompt
                inputs = tokenizer(prompt, return_tensors="pt").to(device)

                # Sinh response từ model hiện tại
                outputs = ppo_model.generate(
                    inputs.input_ids,
                    attention_mask=inputs.attention_mask,
                    max_new_tokens=50,
                    do_sample=True,
                    temperature=0.7,
                    pad_token_id=tokenizer.pad_token_id
                )

                # Decode response
                response_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                response_text = response_text.replace(prompt, "").strip()

                # Tính reward
                reward = reward_function(response_text, chosen, rejected)

                # Chuẩn bị tensors cho batch
                query_tensors.append(tokenizer(prompt, return_tensors="pt").to(device).input_ids)
                response_tensors.append(tokenizer(response_text, return_tensors="pt").to(device).input_ids)
                rewards.append(reward)

                logs["reward"].append(reward)

            try:
                # Chuyển rewards thành tensor
                rewards_tensor = torch.tensor(rewards).to(device)

                # PPO step với batch
                stats = ppo_trainer.step(query_tensors, response_tensors, rewards_tensor)
                logs["policy_loss"].append(stats["policy_loss"])
                logs["value_loss"].append(stats["value_loss"])

                # In thông tin cho batch
                batch_num = i // ppo_config.batch_size + 1
                total_batches = (len(samples) + ppo_config.batch_size - 1) // ppo_config.batch_size

                print(f"\nBatch {batch_num}/{total_batches} (Epoch {epoch+1}):")
                print(f"Reward trung bình: {sum(rewards)/len(rewards):.4f}")
                print(f"Policy Loss: {stats['policy_loss']:.4f}")
                print(f"Value Loss: {stats['value_loss']:.4f}")
                print(f"Thời gian xử lý batch: {time.time() - batch_start_time:.2f}s")

            except Exception as e:
                print(f"Lỗi khi huấn luyện PPO batch {batch_num}: {str(e)}")
                continue

        # In thông tin cho mỗi epoch
        epoch_time = time.time() - epoch_start_time
        avg_reward = sum(logs["reward"][-len(samples):]) / len(samples)
        avg_policy_loss = sum(logs["policy_loss"][-len(batch_samples):]) / len(batch_samples)
        avg_value_loss = sum(logs["value_loss"][-len(batch_samples):]) / len(batch_samples)

        print(f"\nKết thúc epoch {epoch+1}:")
        print(f"Thời gian epoch: {format_time(epoch_time)}")
        print(f"Reward trung bình: {avg_reward:.4f}")
        print(f"Policy Loss trung bình: {avg_policy_loss:.4f}")
        print(f"Value Loss trung bình: {avg_value_loss:.4f}")

    # Tính toán và in thống kê cuối cùng
    total_time = time.time() - total_start_time
    final_avg_reward = sum(logs["reward"]) / len(logs["reward"])

    print(f"\nKết thúc huấn luyện PPO!")
    print(f"Tổng thời gian: {format_time(total_time)}")
    print(f"Reward trung bình cuối: {final_avg_reward:.4f}")

    return ppo_trainer, logs

# Chạy demo PPO với tập mẫu
print("\nBắt đầu chạy PPO demo...")
ppo_trainer, training_logs = run_ppo_demo(demo_samples, num_epochs=1)

print("\nLưu model đã huấn luyện...")
# Lưu model đã huấn luyện
save_path = "ppo_trained_model"
ppo_trainer.save_pretrained(save_path)
print(f"Đã lưu model tại {save_path}")

Khởi tạo PPO config...
Chuyển đổi model sang dạng ValueHead...


/usr/local/lib/python3.11/dist-packages/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


Khởi tạo PPO trainer...

Bắt đầu chạy PPO demo...

Bắt đầu huấn luyện PPO...

Epoch 1/1


UnboundLocalError: cannot access local variable 'batch_num' where it is not associated with a value